## Base Code
4. データの読み込み
5. データのインデックス化
6. 質問に対する類似情報の検索
7. 回答の生成

### Install Libraries

In [1]:
!pip install pdfplumber langchain langchain-openai langchain-chroma langchain-community

### Import Libraries

In [2]:
import requests
import pdfplumber

from langchain.schema import Document
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

from langchain_openai import ChatOpenAI, OpenAIEmbeddings

### Parameters
```python
pdf_file_urls: list
model: str
sample_questions: str
```

In [3]:
model = "gpt-4o-mini"
pdf_file_urls = [
    "https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/Architectural_Design_Service_Contract.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/Call_Center_Operation_Service_Contract.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/Consulting_Service_Contract.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/Content_Production_Service_Contract_(Request_Form).pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/Customer_Referral_Contract.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/Draft_Editing_Service_Contract.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/Graphic_Design_Production_Service_Contract.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/M&A_Advisory_Service_Contract_(Preparatory_Committee).pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/M&A_Intermediary_Service_Contract_SME_M&A_[Small_and_Medium_Enterprises].pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/Manufacturing_Sales_Post-Safety_Management_Contract.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/software_development_outsourcing_contracts.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/Technical_Verification_(PoC)_Contract.pdf",
]

### Load Data
- urlリストからPDFファイルをダウンロードし、Documentオブジェクトのリストとして読み込む

```python
def download_and_load_pdfs(urls: list) -> list
    ...
```

In [4]:
def download_and_load_pdfs(urls: list) -> list:
    """
    PDFファイルをダウンロードして読み込む関数

    Args:
        urls (list): PDFファイルのURLリスト

    Returns:
        documents (list): PDFファイルのテキストデータを含むDocumentオブジェクトのリスト

    Raises:
        Exception: ダウンロードまたは読み込みに失敗した場合に発生する例外

    Examples:
        >>> urls = ["https://example.com/example.pdf"]
        >>> download_and_load_pdfs(urls)
        [Document(page_content="...", metadata={"source": "https://example.com/example.pdf"})]
    """
    try:
        def download_pdf(url, save_path):
            response = requests.get(url)
            if response.status_code == 200:
                with open(save_path, 'wb') as f:
                    f.write(response.content)
            else:
                raise Exception(f"Failed to download {url}")
        documents = []

        for i, url in enumerate(urls):
            tmp_path = f"pdf_{i}.pdf"
            download_pdf(url, tmp_path)

            with pdfplumber.open(tmp_path) as pdf:
                full_text = ""
                for page in pdf.pages:
                    text = page.extract_text()
                    if text:
                        full_text += text + "\n"

                documents.append(
                    Document(
                        page_content=full_text,
                        metadata={"source": url}
                    )
                )
        return documents
    except Exception as e:
        raise Exception(f"Error reading {url}: {e}")

### vector store
- 読み込んだデータに対して、ベクトル検索をするためにインデックス化を行う

```python
def create_vectorstore(docs: list, embeddings) -> Chroma
    ...
```

In [5]:
def create_vectorstore(docs: list, embeddings) -> Chroma:
    """
    テキストデータからベクトルストアを生成する関数

    Args:
        docs (list): Documentオブジェクトのリスト

    Returns:
        vectorstore (Chroma): ベクトルストア

    Raises:
        Exception: ベクトルストアの生成に失敗した場合に発生する例外

    Examples:
        >>> docs = [Document(page_content="...", metadata={"source": "https://example.com/example.pdf"})]
        >>> create_vectorstore(docs)
        Chroma(...)
    """
    try:
        text_splitter = CharacterTextSplitter(
            separator="\n",
            chunk_size=1000,
        )
        splitted_docs = []
        for doc in docs:
            chunks = text_splitter.split_text(doc.page_content)
            for chunk in chunks:
                splitted_docs.append(Document(page_content=chunk, metadata=doc.metadata))

        vectorstore = Chroma.from_documents(
            splitted_docs,
            embeddings,
        )
        return vectorstore
    except Exception as e:
        raise Exception(f"Error creating vectorstore: {e}")

### Pipeline

In [6]:
def rag_implementation(question: str, chat, embeddings) -> str:
    # 4. データの読み込み
    docs = download_and_load_pdfs(pdf_file_urls)

    # 5. データのインデックス化
    db = create_vectorstore(docs, embeddings)

    # 6. 質問に対する類似情報の検索
    retriever = db.as_retriever()

    ### 7. 回答の生成
    template = """
    # ゴール
    私は、参考文章と質問を提供します。
    あなたは、参考文章に基づいて、質問に対する回答を生成してください。

    # 質問
    {question}

    # 参考文章
    {context}
    """
    prompt = ChatPromptTemplate.from_template(template)
    output_parser = StrOutputParser()
    setup_and_retrieval = RunnableParallel(
        {"context": retriever, "question": RunnablePassthrough()}
    )

    chain = setup_and_retrieval | prompt | chat | output_parser
    answer = chain.invoke(question)
    return answer

---
## TEST

### Test Parameters

```python
MockChatModel(Runnable): class
llm: dict
sample_questions: list
```

In [7]:
from langchain.schema.runnable import Runnable
from langchain_community.embeddings import FakeEmbeddings

class MockChatModel(Runnable):
    """ChatOpenAIのモックモデル"""
    def invoke(self, context, config=None, **kwargs):
        return f"Processed: {context}"

llm = {
    "openai": (
        ChatOpenAI(model=model, temperature=0.0),
        OpenAIEmbeddings()
    ),
    "mock": (
        MockChatModel(),
        FakeEmbeddings(size=2048)
    )
}
sample_questions = [
    "ソフトウェア開発業務委託契約について、委託料の金額はいくらですか？",
    "グラフィックデザイン制作業務委託契約について、受託者はいつまでに仕様書を作成して委託者の承諾を得る必要がありますか？",
    "コールセンター業務委託契約における請求書の発行プロセスについて、締め日と発行期限を具体的に説明してください。"
]

### Test Run

In [8]:
# pdfがきちんと読み込めているか確認
docs = download_and_load_pdfs(pdf_file_urls)

# len(urls)==len(docs)か確認
print(f"{len(pdf_file_urls)=}")
print(f"{len(docs)=}")

# doc.page_content が不自然に少ないものはないか確認
print("doc.page_content=[")
for i, doc in enumerate(docs):
    print("  ", end="")
    print(len(doc.page_content), end=",")
    if (i+1)%4 == 0:
        print()
print("]")

len(pdf_file_urls)=12
len(docs)=12
doc.page_content=[
  7748,  6974,  6888,  6426,
  5903,  6372,  5023,  7418,
  4079,  7011,  8275,  7010,
]


In [9]:
question = sample_questions[0]
chat = llm["mock"][0]
embeddings = llm["mock"][1]

result = rag_implementation(question, chat, embeddings)
print(result)

Processed: messages=[HumanMessage(content="\n    # ゴール\n    私は、参考文章と質問を提供します。\n    あなたは、参考文章に基づいて、質問に対する回答を生成してください。\n\n    # 質問\n    存在意義（パーパス）は、なんですか？\n\n    # 参考文章\n    [Document(id='ed2b808c-b924-4739-838c-01fe3528ba9e', metadata={'source': 'https://storage.googleapis.com/gg-raggle-public/competitions/29676d73-5675-4278-b1a6-d4a9fdd0a0ba/dataset/Graphic_Design_Production_Service_Contract.pdf'}, page_content='を受け、若しくはこれらの申立を行ったとき、又は私的整理の開始があったとき。\\n(7) 支払停止、支払不能に陥ったとき。\\n(8) 自ら振出し又は裏書した手形・小切手が一度でも不渡りとなったとき。\\n(9) 資本減少、主要な株主又は取締役の変更、事業譲渡、合併、会社分割等の組織再\\n編その他の会社の支配に重要な影響を及ぼす事実が生じたとき。\\n(10) 公序良俗に反する行為、その他相手方の信用、名誉を毀損する等の背信的行為が\\nあったとき。\\n(11) 解散し、又は事業を廃止したとき。\\n(12) 信用の失墜又はその資産の重大な変動等により、当事者間の信頼関係が損なわれ、\\n本契約の継続が困難であると認める事態が発生したとき。\\n(13) 代表者が刑事上の訴追を受けたとき、又はその所在が不明になったとき。\\n(14) 監督官庁から事業停止処分、又は事業免許若しくは事業登録の取消処分を受けた\\nとき。\\n(15) その他本契約等を継続し難い重大な事由が生じたとき。\\n2. 前項に定める解除は、相手方に対する損害賠償の請求を妨げない。\\n第 11 条 （反社会的勢力の排除）\\n1. 委託者及び受託者は、現在、暴力団、暴力団員、暴力団員でなくなった時から 5 年を\\n経過しない者、暴力団準構成員、暴力団関係企業、総会屋等、社